In [7]:
import numpy as np
import matplotlib.pyplot as plt
%pylab inline
delta=5
def gausin_distance(x,y,delta=delta):
    H= torch.norm(x-y).cuda()
    distance = torch.exp(-H/2/(delta**2)).cuda()
    
    return distance
def gram_matrix(data,kernel=gausin_distance):
    #num=data.shape[0]
    #matrix=torch.zeros((num,num)).cuda()
    #for i in range (0,num):
    #   for j in range (0,num):
    #        matrix[i][j]=kernel(data[i],data[j])
    num=data.shape[0]
    datav3=torch.mm(data,torch.transpose(data, 0, 1).cuda()).cuda()
    datav1=torch.diag(datav3,0).cuda()
    #print(datav1.shape)
    #matrix=torch.sqrt(datav1+datav2-2*datav3)
    buf1=((-2)*datav3+datav1).cuda()
    buf2=torch.transpose(buf1, 0, 1).cuda()
    buf3=(buf2+datav1).cuda()
    #print(buf3)
    matrix = torch.exp(-buf3/2/(delta**2)).cuda()
    return matrix
def emerinal_hsic(X,Y):
    num=X.shape[0]
    #print(num)
    
    
    
    Kx=gram_matrix(X)
    
    Ky=gram_matrix(Y)
    
    H = torch.eye(num).cuda() - torch.ones((num,num), dtype = torch.float32).cuda() / num
    #print(H)
    hsic=1/(num-1)*torch.trace(torch.mm(torch.mm(torch.mm(Kx,H).cuda(),Ky).cuda(),H).cuda()).cuda()
    
    return hsic
import torch
import torchvision
from torchvision import transforms
import torch.nn.functional as F
import torchvision.datasets
from PIL import Image
batch_size=256
class MyMNIST(torchvision.datasets.MNIST):
    def __init__(self, root, train=True, transform=None, target_transform=None,download=False):
        
        super(MyMNIST,self).__init__(root, train, transform, target_transform,download)
        #print(self.data)
    def __getitem__(self, index):
        """
        Args:
            index (int): Index

        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        img, target = self.data[index], int(self.targets[index])
        #print(target) 
        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = Image.fromarray(img.numpy(), mode='L')

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)
            
        else:
            target=torch.as_tensor(target)
            target = F.one_hot(target,num_classes=10).cuda()

        return img, target
mnist_data=torchvision.datasets.MNIST(root='./MNIST/', train=True, transform=transforms.ToTensor(), target_transform=None, download=False)
data_loader = torch.utils.data.DataLoader(mnist_data,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=2)
#D_in, H1,H2,H3,H4,H5,H6,H7, D_out = 784, 64,32,16,8,4,2,1,10
layer_num=[784,64]
costs=[]
dtype=torch.float
continue_flag=0
PATH='./checkpoint_cnn_onelayer'
w=[]
z=[]
rate_list=[]
if continue_flag:
    
    checkpoint = torch.load(PATH)
    
    w=checkpoint['w']

else:
    for i in range(len(layer_num)-1):
        m = torch.nn.Conv2d(1, 2, 3, stride=1).cuda()
        w.append(m)

epoches=20
lamda=5
learning_rate=0.1
data_tsne_label=[]
data_tsne_rp1=[]
data_tsne_rp2=[]
for i in range(epoches): 
    for i_batch, (img,target) in enumerate(data_loader):
        z=[]
        img=img.reshape(-1,1,28,28).cuda()
        
        target=torch.as_tensor(target).cuda()
        num=target.shape[0]
        #print(num)
        target=target.reshape(num,1).cuda()
        #target = F.one_hot(target,num_classes=10).cuda()
        target=torch.zeros((num,10)).cuda().scatter_(1,target,1).cuda()
        
        target=torch.as_tensor(target,dtype=dtype).cuda()
        
        z.append(F.relu(F.max_pool2d(w[0](img), 2)).cuda())
        for cnt in range(len(w)-1):
            
            z.append(torch.tanh(torch.mm(z[cnt],w[cnt+1]).cuda()).cuda())
            
        grad_list=[]
        
        for cnt in range(len(z)-1):

            
            cost=-emerinal_hsic(z[cnt],img)-lamda*emerinal_hsic(z[cnt],target)
        #print(cost1)
            cost.backward(retain_graph=True)
            print('!')
        #print(w1.cuda().grad)
            with torch.no_grad():
                buf=torch.zeros((w[cnt].shape)).cuda()
                #print(buf.shape)
                buf.copy_(w[cnt].grad)
                
                grad_list.append(buf)
            #print('grad1',torch.norm(grad1))
            #w2 -= learning_rate * w2.grad

            # Manually zero the gradients after updating weights
                w[cnt].grad.zero_()
                #print(cnt,torch.norm(buf))
            #w2.grad.zero_()
        z[-1]=z[-1].view(img.shape[0],-1).cuda()
        print(z[-1].shape)
        img=img.view(img.shape[0],-1).cuda()
        #print(z[-1].shape)
        entropy=emerinal_hsic(z[-1],img)
        information=lamda*emerinal_hsic(z[-1],target)
        rate=entropy/information
        cost=-entropy-information+1*(entropy/information)
        cost.backward()
        
        with torch.no_grad():
            #w1 -= learning_rate * w1.grad
            #print('grad2',torch.norm(w2.grad))
            w[-1].weight -= learning_rate * w[-1].weight.grad
            
            for cnt in range(len(w)-1):
                w[cnt] -= learning_rate * grad_list[cnt]
        # Manually zero the gradients after updating weights
            w[-1].weight.grad.zero_()
            
        #    data_tsne_rp2.append(z2.cpu().data.numpy())
        #    data_tsne_rp1.append(z1.cpu().data.numpy())
        #    data_tsne_label.append(target.argmax(dim=1, keepdim=True).cpu().data.numpy())
        

        if i_batch %100 ==0:
            costs.append(cost)
            print(rate)
            rate_list.append(rate)
            print(i_batch)
            print('cost',cost)
            print('entr',entropy)
            print('inf',information)
            data_tsne_label.append(target.argmax(dim=1, keepdim=True).cpu().data.numpy())
            torch.save({
            'w': w,
            'z': z,
            'data_tsne_label':target.argmax(dim=1, keepdim=True).cpu().data.numpy()
            ,'costs':costs
            #'data_tsne_rp2':data_tsne_rp2,
            #'data_tsne_rp1':data_tsne_rp1
                
            }, PATH)
            
            data_tsne_label=[]
            #data_tsne_rp1=[]
            #data_tsne_rp2=[]

            #for j,zz in enumerate(z2):
                #print(j,zz)
                #print(j,target[j])
            
plt.plot(rate_list)
plt.ylabel('rate')
plt.xlabel('epochs (per 1)')
plt.title("Learning rate = " + str(learning_rate))
plt.show()

Populating the interactive namespace from numpy and matplotlib
torch.Size([256, 338])
tensor(3.0950, device='cuda:0', grad_fn=<DivBackward0>)
0
cost tensor(2.9111, device='cuda:0', grad_fn=<AddBackward0>)
entr tensor(0.1390, device='cuda:0', grad_fn=<MulBackward0>)
inf tensor(0.0449, device='cuda:0', grad_fn=<MulBackward0>)
torch.Size([256, 338])
torch.Size([256, 338])
torch.Size([256, 338])
torch.Size([256, 338])
torch.Size([256, 338])
torch.Size([256, 338])
torch.Size([256, 338])
torch.Size([256, 338])
torch.Size([256, 338])
torch.Size([256, 338])
torch.Size([256, 338])
torch.Size([256, 338])
torch.Size([256, 338])
torch.Size([256, 338])
torch.Size([256, 338])
torch.Size([256, 338])
torch.Size([256, 338])
torch.Size([256, 338])
torch.Size([256, 338])
torch.Size([256, 338])
torch.Size([256, 338])
torch.Size([256, 338])
torch.Size([256, 338])
torch.Size([256, 338])
torch.Size([256, 338])
torch.Size([256, 338])
torch.Size([256, 338])
torch.Size([256, 338])
torch.Size([256, 338])
torch.S

KeyboardInterrupt: 